In [1]:
import applySPI as ap
import dill
import pyspi
import loadData as ld
import random
import numpy as np
import collections
import pandas as pd
from tqdm import tqdm
import sys
import os
from sklearn.metrics import roc_auc_score
from sklearn.cluster import SpectralClustering
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, adjusted_mutual_info_score, homogeneity_score, completeness_score, v_measure_score
import sys

In [2]:
# https://stackoverflow.com/a/45669280
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [16]:
with HiddenPrints():
    calc = pyspi.calculator.Calculator(subset='fast')

# specify the location of your saved calculator .pkl file
loc = r'.\saved_calculator_name.pkl'
# loc = r'.\saved_calculator_wo_data.pkl'

with open(loc, 'rb') as f:
    calc = dill.load(f)

Frequency minimum set to 0; overriding to 1e-5.


In [18]:
# get the dataset
dataset, _, _ = ld.load_keti(path=r"C:\Users\Lucas\Data\KETI", sample_rate='1min')
dataset = dataset[dataset.columns[:35]]
print(f'We have {dataset.shape[1]} signals with {dataset.shape[0]} samples per signal.')

Rename and Reformat: 100%|██████████| 51/51 [00:00<00:00, 943.53it/s]

Loaded and resampled all with sampling rate 1min signals from (Timestamp('2013-08-25 15:22:00'), Timestamp('2013-09-01 06:58:00')) with length 9577.
We have 35 signals with 9577 samples per signal.


In [19]:
calc._dataset = None
# save the calculator object as a .pkl
with open('saved_calculator_wo_data.pkl', 'wb') as f:
    dill.dump(calc, f)

In [20]:
# rename the rooms
calc.table.index = dataset.columns
first_table = list(calc.table.columns.get_level_values(0))[0]
calc.table.rename(columns={level_two: col for (level_two, col) in zip(calc.table[first_table].columns, dataset.columns)}, inplace=True)

In [21]:
# get the rooms
rooms = {col.split('_')[0] for col in dataset.columns}
sensors = {col.split('_')[1] for col in dataset.columns}

In [22]:
# get all the measures we want and that have only main diagonal NaN
measures = set(calc.table.columns.get_level_values(0))
original_amount = len(measures)
measures = [table for table in measures if calc.table[table].shape[1] == calc.table[table].isna().to_numpy().sum()]
print(f'We retained {len(measures)}/{original_amount} similarity measures.')

# create dataframe to save results
results = pd.DataFrame(index=measures, columns=["gross accuracy", "median rank", "pairwise auroc", "Adjusted Rand Index", "Normalized Mutual Information", "Adjusted Mutual Information", "Homogeneity", "Completeness", "V-Measure", 'Triplet Accuracy'], dtype=float)

# produce the results
for table in tqdm(measures, desc='Going through measures'):
    gross_accuracy = 0
        
    # drop from the index, so we can't use it to compare
    currtab = calc.table[table].copy()
    results.loc[table, "gross accuracy"] = 0
    
    # check whether we have too many nan values
    if currtab.isna().to_numpy().sum() > currtab.shape[1]:
        print(table, 'has NaN.')
        continue
    
    # go through the sensor and check the k closest
    correct = 0
    for element in currtab.columns:
        
        # drop the own element
        series = currtab[element].drop(index=element)
        
        # go to the corresponding column and check the closest elements
        closest = series.nlargest(1)
        
        # check majority class and also the closest element
        rooms = dict()
        for index, content in closest.items():
            room = index.split('_')[0]
            if room not in rooms:
                rooms[room] = [1, content]
            else:
                rooms[room][0] += 1
                rooms[room][1] = max(content, rooms[room][1])
        
        # get the maximum element
        max_ele = max(rooms.items(), key= lambda x: x[1])[0]
        gross_accuracy += max_ele == element.split('_')[0]
    results.loc[table, "gross accuracy"] = gross_accuracy/currtab.shape[1]

We retained 187/216 similarity measures.


Going through measures: 100%|██████████| 187/187 [00:04<00:00, 43.59it/s]


In [8]:
results["gross accuracy"].nlargest(10)

prec-sq_LedoitWolf                             0.600000
prec-sq_ShrunkCovariance                       0.600000
phase_multitaper_mean_fs-1_fmin-0_fmax-0-25    0.571429
phase_multitaper_mean_fs-1_fmin-0_fmax-0-5     0.571429
prec-sq_OAS                                    0.542857
prec-sq_EmpiricalCovariance                    0.542857
si_gaussian_k-1                                0.542857
pec                                            0.514286
prec_LedoitWolf                                0.514286
wpli_multitaper_mean_fs-1_fmin-0_fmax-0-25     0.485714
Name: gross accuracy, dtype: float64

In [23]:
results["gross accuracy"].nlargest(10)

prec-sq_ShrunkCovariance                       0.600000
prec-sq_LedoitWolf                             0.600000
phase_multitaper_mean_fs-1_fmin-0_fmax-0-25    0.571429
phase_multitaper_mean_fs-1_fmin-0_fmax-0-5     0.571429
prec-sq_OAS                                    0.542857
si_gaussian_k-1                                0.542857
prec-sq_EmpiricalCovariance                    0.542857
prec_LedoitWolf                                0.514286
pec                                            0.514286
wpli_multitaper_mean_fs-1_fmin-0_fmax-0-25     0.485714
Name: gross accuracy, dtype: float64

In [24]:
# produce the results
for table in tqdm(measures, desc='Going through measures'):
    
    # get a copy of the results
    currtab = calc.table[table].copy()
    
    # go through all columns and check the rank of other sensors from the same room
    ranks = 0
    for col in currtab.columns:
        
        # Extract the room number and type from the original string
        room, type_to_exclude = col.split('_')
        
        # get the median rank of the sensors
        median_rank = currtab[col].rank(ascending=False).filter(regex=rf'^{room}_(?!{type_to_exclude}$)').median()
        ranks += median_rank
    results.loc[table, "median rank"] = -ranks/len(currtab.columns)

Going through measures: 100%|██████████| 187/187 [00:02<00:00, 88.94it/s]


In [10]:
results["median rank"].nlargest(10)

si_gaussian_k-1                                -10.057143
prec-sq_EmpiricalCovariance                    -11.100000
prec-sq_OAS                                    -11.114286
prec-sq_LedoitWolf                             -11.700000
prec-sq_ShrunkCovariance                       -11.857143
plv_multitaper_mean_fs-1_fmin-0_fmax-0-25      -11.871429
ppc_multitaper_mean_fs-1_fmin-0_fmax-0-25      -12.042857
coint_johansen_max_eig_stat_order-0_ardiff-1   -12.342857
coint_johansen_max_eig_stat_order-1_ardiff-1   -12.342857
cohmag_multitaper_max_fs-1_fmin-0_fmax-0-25    -12.442857
Name: median rank, dtype: float64

In [25]:
results["median rank"].nlargest(10)

si_gaussian_k-1                                -10.057143
prec-sq_EmpiricalCovariance                    -11.100000
prec-sq_OAS                                    -11.114286
prec-sq_LedoitWolf                             -11.700000
prec-sq_ShrunkCovariance                       -11.857143
plv_multitaper_mean_fs-1_fmin-0_fmax-0-25      -11.871429
ppc_multitaper_mean_fs-1_fmin-0_fmax-0-25      -12.042857
coint_johansen_max_eig_stat_order-0_ardiff-1   -12.342857
coint_johansen_max_eig_stat_order-1_ardiff-1   -12.342857
cohmag_multitaper_max_fs-1_fmin-0_fmax-0-25    -12.442857
Name: median rank, dtype: float64

In [11]:
# now we can also use auroc to for pairwise interactions
for table in tqdm(measures, desc='Going through measures'):
    
    # get a copy of the results
    currtab = calc.table[table].copy()
    
    # make a copy to create the groundtruth
    gt = currtab.copy()
    gt.loc[:, :] = 0
    
    # make a numpy array that shows the rooms
    first_letters = np.array([int(ele[0]) for ele in gt.index.str.split('_')])
    
    # go through the columns of the ground truth and place the ground truth
    for col in gt.columns:
        room = int(col.split('_')[0])
        gt.loc[first_letters == room, col] = 1
    
    # Create a mask for the main diagonal
    diagonal_mask = np.eye(currtab.shape[0], dtype=bool)
    
    # Invert the mask to get the non-diagonal elements
    non_diagonal_mask = ~diagonal_mask
    
    # Flatten the DataFrame using the mask
    flattened_non_diagonal = gt.values[non_diagonal_mask]
    
    # compute the roc_auc
    roc_auc = roc_auc_score(gt.values[non_diagonal_mask], currtab.values[non_diagonal_mask])
    
    # put in the auc score
    results.loc[table, "pairwise auroc"] = roc_auc

Going through measures: 100%|██████████| 187/187 [00:01<00:00, 168.68it/s]


In [12]:
results["pairwise auroc"].nlargest(10)

ppc_multitaper_mean_fs-1_fmin-0_fmax-0-25      0.668816
plv_multitaper_mean_fs-1_fmin-0_fmax-0-25      0.666857
si_gaussian_k-1                                0.654857
dspli_multitaper_mean_fs-1_fmin-0_fmax-0-25    0.653197
ppc_multitaper_max_fs-1_fmin-0_fmax-0-5        0.640000
plv_multitaper_max_fs-1_fmin-0_fmax-0-5        0.640000
ppc_multitaper_mean_fs-1_fmin-0_fmax-0-5       0.638639
plv_multitaper_mean_fs-1_fmin-0_fmax-0-5       0.636272
plv_multitaper_max_fs-1_fmin-0_fmax-0-25       0.630857
ppc_multitaper_max_fs-1_fmin-0_fmax-0-25       0.630857
Name: pairwise auroc, dtype: float64

In [13]:
# now we can also use auroc to for pairwise interactions
rooms = {col.split('_')[0] for col in dataset.columns}
for table in tqdm(measures, desc='Going through measures'):
    
    # get a copy of the results
    currtab = calc.table[table].copy()
    currtab.loc[:, :] -= currtab.min().min()
    
    # Compute the symmetric DataFrame by averaging with its transpose
    currtab = (currtab + currtab.T) / 2
    
    # fill the main diagonal of the dataframe
    np.fill_diagonal(currtab.values, 1)
    
    # make some clustering
    clustering = SpectralClustering(n_clusters=len(rooms), affinity='precomputed')
    predicted_labels = clustering.fit_predict(currtab)
    
    # evaluate the clustering
    gt = [int(col.split('_')[0]) for col in currtab.columns]
    
    ari_score = adjusted_rand_score(gt, predicted_labels)
    results.loc[table, "Adjusted Rand Index"] = ari_score
    
    nmi_score = normalized_mutual_info_score(gt, predicted_labels)
    results.loc[table, "Normalized Mutual Information"] = nmi_score
    
    ami_score = adjusted_mutual_info_score(gt, predicted_labels)
    results.loc[table, "Adjusted Mutual Information"] = ami_score
    
    homogeneity = homogeneity_score(gt, predicted_labels)
    results.loc[table, "Homogeneity"] = homogeneity
    
    completeness = completeness_score(gt, predicted_labels)
    results.loc[table, "Completeness"] = completeness
    
    v_measure = v_measure_score(gt, predicted_labels)
    results.loc[table, "V-Measure"] = v_measure

Going through measures: 100%|██████████| 187/187 [00:04<00:00, 46.19it/s]


In [14]:
results["Adjusted Rand Index"].nlargest(10)

psi_wavelet_mean_fs-1_fmin-0_fmax-0-5_mean        0.605774
psi_wavelet_mean_fs-1_fmin-0-25_fmax-0-5_mean     0.605774
psi_wavelet_max_fs-1_fmin-0_fmax-0-25_max         0.558351
psi_wavelet_mean_fs-1_fmin-0_fmax-0-25_mean       0.558351
psi_wavelet_max_fs-1_fmin-0_fmax-0-5_max          0.541410
psi_wavelet_max_fs-1_fmin-0-25_fmax-0-5_max       0.541410
coint_aeg_tstat_trend-ct_autolag-bic_maxlag-10    0.432681
gc_gaussian_k-1_kt-1_l-1_lt-1                     0.230552
si_gaussian_k-1                                   0.225241
coint_johansen_max_eig_stat_order-1_ardiff-1      0.178828
Name: Adjusted Rand Index, dtype: float64

In [15]:
results["Normalized Mutual Information"].nlargest(10)

psi_wavelet_mean_fs-1_fmin-0_fmax-0-5_mean        0.777774
psi_wavelet_mean_fs-1_fmin-0-25_fmax-0-5_mean     0.777774
psi_wavelet_max_fs-1_fmin-0_fmax-0-25_max         0.772704
psi_wavelet_mean_fs-1_fmin-0_fmax-0-25_mean       0.772704
psi_wavelet_max_fs-1_fmin-0_fmax-0-5_max          0.740983
psi_wavelet_max_fs-1_fmin-0-25_fmax-0-5_max       0.740983
coint_aeg_tstat_trend-ct_autolag-bic_maxlag-10    0.712305
gc_gaussian_k-1_kt-1_l-1_lt-1                     0.544297
coint_johansen_max_eig_stat_order-1_ardiff-1      0.534934
si_gaussian_k-1                                   0.530503
Name: Normalized Mutual Information, dtype: float64

In [16]:
results["Homogeneity"].nlargest(10)

psi_wavelet_mean_fs-1_fmin-0_fmax-0-5_mean        0.776624
psi_wavelet_mean_fs-1_fmin-0-25_fmax-0-5_mean     0.776624
psi_wavelet_max_fs-1_fmin-0_fmax-0-25_max         0.742909
psi_wavelet_mean_fs-1_fmin-0_fmax-0-25_mean       0.742909
psi_wavelet_max_fs-1_fmin-0_fmax-0-5_max          0.739888
psi_wavelet_max_fs-1_fmin-0-25_fmax-0-5_max       0.739888
coint_aeg_tstat_trend-ct_autolag-bic_maxlag-10    0.668789
si_gaussian_k-1                                   0.519007
gc_gaussian_k-1_kt-1_l-1_lt-1                     0.502466
coint_johansen_max_eig_stat_order-1_ardiff-1      0.496648
Name: Homogeneity, dtype: float64

In [17]:
results["Adjusted Mutual Information"].nlargest(10)

psi_wavelet_max_fs-1_fmin-0_fmax-0-25_max         0.669337
psi_wavelet_mean_fs-1_fmin-0_fmax-0-25_mean       0.669337
psi_wavelet_mean_fs-1_fmin-0_fmax-0-5_mean        0.667968
psi_wavelet_mean_fs-1_fmin-0-25_fmax-0-5_mean     0.667968
psi_wavelet_max_fs-1_fmin-0_fmax-0-5_max          0.612998
psi_wavelet_max_fs-1_fmin-0-25_fmax-0-5_max       0.612998
coint_aeg_tstat_trend-ct_autolag-bic_maxlag-10    0.589310
gc_gaussian_k-1_kt-1_l-1_lt-1                     0.349117
coint_johansen_max_eig_stat_order-1_ardiff-1      0.321312
coint_aeg_tstat_trend-c_autolag-aic_maxlag-10     0.311030
Name: Adjusted Mutual Information, dtype: float64

In [18]:
# make the triplet accuracy
for table in tqdm(measures, desc='Going through measures'):
    
    # get a copy of the results
    currtab = calc.table[table].copy()
    
    # go through all the different triplets and make accuracy
    sensors = list(currtab.columns)
    
    # make all the triplets and whether they are successful
    triplets = [currtab.loc[anchor, positive] > currtab.loc[anchor, negative] for adx, anchor in enumerate(sensors) for pdx, positive in enumerate(sensors) for ndx, negative in enumerate(sensors) if ndx != pdx and ndx != adx and pdx != adx and anchor.split('_')[0] == positive.split('_')[0] and anchor.split('_')[0] != negative.split('_')[0]]
    
    # compute the triplet accuracy
    results.loc[table, 'Triplet Accuracy'] = sum(triplets)/len(triplets)

Going through measures: 100%|██████████| 187/187 [00:16<00:00, 11.47it/s]


In [19]:
results['Triplet Accuracy'].nlargest(10)

si_gaussian_k-1                                 0.700238
plv_multitaper_mean_fs-1_fmin-0_fmax-0-25       0.678571
ppc_multitaper_mean_fs-1_fmin-0_fmax-0-25       0.677381
prec-sq_EmpiricalCovariance                     0.665714
prec-sq_OAS                                     0.657857
dspli_multitaper_mean_fs-1_fmin-0_fmax-0-25     0.653571
prec-sq_ShrunkCovariance                        0.648810
prec-sq_LedoitWolf                              0.643333
coint_johansen_max_eig_stat_order-0_ardiff-1    0.638333
coint_johansen_max_eig_stat_order-1_ardiff-1    0.638333
Name: Triplet Accuracy, dtype: float64

In [20]:
results.rank(ascending=False).mean(axis=1).nsmallest(10)

si_gaussian_k-1                                   7.40
gc_gaussian_k-1_kt-1_l-1_lt-1                    13.75
pec                                              14.40
cohmag_multitaper_max_fs-1_fmin-0_fmax-0-25      21.35
pec_orth                                         22.00
dspli_multitaper_mean_fs-1_fmin-0_fmax-0-25      22.80
dspli_multitaper_mean_fs-1_fmin-0_fmax-0-5       25.30
coint_johansen_max_eig_stat_order-1_ardiff-1     26.20
coint_johansen_max_eig_stat_order-1_ardiff-10    26.90
coint_johansen_max_eig_stat_order-0_ardiff-1     28.00
dtype: float64

In [21]:
results.rank(ascending=False).loc["si_gaussian_k-1"]

gross accuracy                    6.0
median rank                       1.0
pairwise auroc                    3.0
Adjusted Rand Index               9.0
Normalized Mutual Information    10.0
Adjusted Mutual Information      11.0
Homogeneity                       8.0
Completeness                     15.0
V-Measure                        10.0
Triplet Accuracy                  1.0
Name: si_gaussian_k-1, dtype: float64